#Data Question 4: Web Scraping Earthquake Data
##Part 1:

Nate Silver discusses the difficulty of predicting earthquakes in The Signal and the Noise. Nevertheless, we will try to identify some patterns by analyzing the deadly earthquakes that have occurred since 1900.

To start, read the table of earthquakes from https://en.wikipedia.org/wiki/List_of_deadly_earthquakes_since_1900 using the requests and/or beautifulsoup library and load it to a pandas dataframe. You will need to do some data cleaning before you can proceed.

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_deadly_earthquakes_since_1900'


##Data cleaning tasks include:

- Replace empty strings with NaN
- Remove the footnotes from the 'Other Source Deaths' column
- Convert Magnitude to a numeric type. For this portion, you can ignore differences in seismic magnitude scales.
- Correct number of deaths when there is more than one value. When there is more than one value given, choose the largest.
- Create a new column ('deaths') that evaluates the four total-death columns ('PDE Total Deaths', 'Utsu Total Deaths', 'EM-DAT Total Deaths', and 'Other Source Deaths') and populates the new column with the highest value.
- Explore the data in terms of when and where earthquakes occurred and how severe they were (magnitude, deaths, secondary effects).